In [0]:
%sql
select * from delta.`/mnt/bronze/multiplexbronzetable` where source='supporttickets';

In [0]:
df=spark.readStream.format('delta').load("/mnt/bronze/multiplexbronzetable").filter("source=='supporttickets'")

In [0]:
from pyspark.sql.types import *

support_schema = StructType([
    StructField("ticket_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("order_id", StringType(), True),
    StructField("created_at", TimestampType(), True),
    StructField("resolved_at", TimestampType(), True),
    StructField("status", StringType(), True),
    StructField("issue_type", StringType(), True),
    StructField("priority", StringType(), True),
    StructField("agent", StringType(), True),
    StructField("channel", StringType(), True),
    StructField("satisfaction_rating", IntegerType(), True)
])

In [0]:
from pyspark.sql.functions import from_json,cast,col

parsed_df=df.withColumn('payload',from_json(col('raw_payload').cast('String'),support_schema)).select('payload.*','ingesttimestamp')
deduped_df=parsed_df\
        .withWatermark("ingesttimestamp", "10 hours")\
        .dropDuplicates(["ticket_id"])


In [0]:
display(deduped_df)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS support_tickets_silver (
  ticket_id STRING,
  customer_id STRING,
  order_id STRING,
  created_at TIMESTAMP,
  resolved_at TIMESTAMP,
  status STRING,
  issue_type STRING,
  priority STRING,
  agent STRING,
  channel STRING,
  satisfaction_rating INT,
  ingesttimestamp TIMESTAMP
)
USING DELTA
PARTITIONED BY (ingesttimestamp)
LOCATION '/mnt/silver/support_tickets_silver';

In [0]:
deduped_df.writeStream\
    .format("delta")\
        .option("checkpointLocation", "/mnt/checkpoints/silver/support_tickets")\
        .option("mergeSchema",'true')\
            .partitionBy('ingesttimestamp')\
                .table('support_tickets_silver')

In [0]:
%sql
select * from support_tickets_silver;